In [1]:
from scipy import stats
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas_profiling as pp
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import precision_score, roc_auc_score, make_scorer, f1_score, recall_score, roc_auc_score
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler
import os
os.environ['PYTHONHASHSEED']=str(271828)
random.seed(271828)
np.random.seed(271828)
#import tensorflow as tf
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from tensorflow.python.framework import ops
import tensorflow as tf
tf.random.set_seed(271828)
ops.reset_default_graph()
from fancyimpute import IterativeImputer

from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
#from tensorflow.keras.layers import Conv2D, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import EarlyStopping

import keras.backend as K
#session_conf=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1) 
#sesCriticalSection=tf.Session(graph=tf.get_default_graph(), config=session_conf)
#K.set_session(sess)

Using TensorFlow backend.


In [2]:
def PCL_calculator(df):

    symptomatic_cutoff = 2

    intrusion = ['q6.1_INTRU', 'q6.2_DREAM', 'q6.3_FLASH', 'q6.4_UPSET', 'q6.5_PHYS']
    avoidance = ['q6.6_AVTHT', 'q6.7_AVSIT', 'q6.8_AMNES', 'q6.9_DISINT', 'q6.10_DTACH',
                 'q6.11_NUMB', 'q6.12_FUTRE']
    tred = ['q6.1_INTRU', 'q6.2_DREAM', 'q6.3_FLASH']
    only_avoidance = ['q6.6_AVTHT', 'q6.7_AVSIT', 'q6.8_AMNES']

    hypertension = ['q6.13_SLEEP', 'q6.14_ANGER', 'q6.15_CONC', 'q6.16_HYPER', 'q6.17_STRTL']

    depression = ['q6.9_DISINT', 'q6.10_DTACH', 'q6.11_NUMB', 'q6.12_FUTRE']

    df[intrusion + avoidance + hypertension].fillna(df[intrusion + avoidance + hypertension].mean(axis=1))
    intrusion_cuoff = 1
    avoidance_cuoff = 3
    hypertension_cuoff = 2
    only_avoidance_cutoff = 1
    depression_cutoff = 2
    tred_cutoff = 1

    df['sum'] = (df[intrusion + avoidance + hypertension]).sum(axis=1)

    df['intrusion'] = (df[intrusion] > symptomatic_cutoff).sum(axis=1)
    df['intrusion_cutoff'] = (df['intrusion'] >= intrusion_cuoff).astype(int)

    df['avoidance'] = (df[avoidance] > symptomatic_cutoff).sum(axis=1)
    df['avoidance_cutoff'] = (df['avoidance'] >= avoidance_cuoff).astype(int)

    df['depression'] = (df[depression] > symptomatic_cutoff).sum(axis=1)
    df['depression_cutoff'] = df['depression'] >= depression_cutoff

    df['hypertention'] = (df[hypertension] > symptomatic_cutoff).sum(axis=1)
    df['hypertention_cutoff'] = (df['hypertention'] >= hypertension_cuoff).astype(int)

    df['tred'] = (df[tred] > symptomatic_cutoff).sum(axis=1)
    df['tred_cutoff'] = df['tred'] >= tred_cutoff

    df['only_avoidance'] = (df[only_avoidance] > symptomatic_cutoff).sum(axis=1)
    df['only_avoidance_cutoff'] = df['only_avoidance'] >= only_avoidance_cutoff

    df['regression_cutoff_33'] = df['sum'] >= 33
    df['regression_cutoff_50'] = df['sum'] >= 49
    df['diagnosis'] = ((df['hypertention_cutoff']) & (df['avoidance_cutoff']) & (df['intrusion_cutoff']) & (df['sum'] >= 49))

    return df


In [3]:
path = "C:\‏‏PycharmProjects\PTSD\Data\PTSD.xlsx"
df = pd.read_excel(path)
#combine with specifics of answers
df_pcl2 = pd.read_excel("C:\‏‏PycharmProjects\PTSD\Data\questionnaire6PCL2.xlsx")
df_pcl1 = pd.read_excel("C:\‏‏PycharmProjects\PTSD\Data\questionnaire6PCL1.xlsx")
df = df.merge(df_pcl1, on="ID", how='outer')
df = df.merge(df_pcl2, suffixes=('_pcl1', '_pcl2'), on="ID", how='outer')

df_pcl3 = pd.read_excel("C:\‏‏PycharmProjects\PTSD\Data\questionnaire6PCL3.xlsx")
df_pcl3 = PCL_calculator(df_pcl3)
df = df.merge(df_pcl3.drop(['PCL3_Strict', 'pcl3', 'PCL3_Broad'], axis=1), on="ID", how='outer')
# rmoving missing Y's
df = df[~df['tred_cutoff'].isna()]

In [4]:
# removed age
features = ["highschool_diploma", "dyslexia", "ADHD", "Ashken_scale", "Sephar_scale", "Unknown_scale",
            "T1Acc1t", "T1Acc1n", "T1bias", "phq1", "lot1", "trait1", "state1", "PCL1",
            "PCL_Broad1", "PCL_Strict1", "phq2", "lot2", "trait2", "state2", "PCL2",
            "PCL_Broad2", "PCL_Strict2", "cd_risc1", "active_coping1", "planning1",
            "positive_reframing1", "acceptance1", "humor1", "religion1", "emotional_support1",
            "instrumental_support1", "self_distraction1", "denial1", "venting1", "substance_use1",
            "behavioral_disengagement1", "self_blame1", "active_coping2", "planning2",
                "positive_reframing2", "acceptance2", "humor2", "religion2", "emotional_support2",
                "instrumental_support2", "self_blame2", "self_distraction2", "denial2",
            "venting2", "substance_use2", "behavioral_disengagement2",
                "trauma_history8_1", "HML_5HTT", "HL_MAOA", "HML_NPY", "COMT_Hap1_recode",
                "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", 'q6.1_INTRU_pcl1', 
                'q6.2_DREAM_pcl1', 'q6.3_FLASH_pcl1', 'q6.4_UPSET_pcl1',
                  'q6.5_PHYS_pcl1', 'q6.6_AVTHT_pcl1', 'q6.7_AVSIT_pcl1', 'q6.8_AMNES_pcl1', 'q6.9_DISINT_pcl1',
                  'q6.10_DTACH_pcl1', 'q6.11_NUMB_pcl1', 'q6.12_FUTRE_pcl1', 'q6.13_SLEEP_pcl1',
                  'q6.14_ANGER_pcl1', 'q6.15_CONC_pcl1', 'q6.16_HYPER_pcl1', 'q6.17_STRTL_pcl1',
                  'q6.1_INTRU_pcl2', 'q6.2_DREAM_pcl2', 'q6.3_FLASH_pcl2', 'q6.4_UPSET_pcl2',
                  'q6.5_PHYS_pcl2', 'q6.6_AVTHT_pcl2', 'q6.7_AVSIT_pcl2', 'q6.8_AMNES_pcl2', 'q6.9_DISINT_pcl2',
                  'q6.10_DTACH_pcl2', 'q6.11_NUMB_pcl2', 'q6.12_FUTRE_pcl2', 'q6.13_SLEEP_pcl2',
                  'q6.14_ANGER_pcl2', 'q6.15_CONC_pcl2', 'q6.16_HYPER_pcl2', 'q6.17_STRTL_pcl2']

In [5]:
t2_features = [
    "lot2", "trait2", "state2", "PCL2", "PCL_Broad2", "PCL_Strict2", "phq2",

    "active_coping2", "planning2", "positive_reframing2", "acceptance2", "humor2", 
    "religion2", "emotional_support2", "instrumental_support2", "self_distraction2",
    "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
    
    'q6.1_INTRU_pcl2', 'q6.2_DREAM_pcl2', 'q6.3_FLASH_pcl2', 'q6.4_UPSET_pcl2', 'q6.5_PHYS_pcl2',
    'q6.6_AVTHT_pcl2', 'q6.7_AVSIT_pcl2', 'q6.8_AMNES_pcl2', 'q6.9_DISINT_pcl2',
                    'q6.10_DTACH_pcl2', 'q6.11_NUMB_pcl2', 'q6.12_FUTRE_pcl2', 'q6.13_SLEEP_pcl2',
                    'q6.14_ANGER_pcl2', 'q6.15_CONC_pcl2', 'q6.16_HYPER_pcl2', 'q6.17_STRTL_pcl2'
]
t1_features =[ "phq1", "lot1", "trait1","state1", "PCL1", "PCL_Broad1", "PCL_Strict1",
              
                    "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1", "religion1",
                    "emotional_support1", "instrumental_support1", "self_distraction1", "denial1",
                    "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1",
                'q6.1_INTRU_pcl1', 'q6.2_DREAM_pcl1', 'q6.3_FLASH_pcl1', 'q6.4_UPSET_pcl1',
                    'q6.5_PHYS_pcl1', 'q6.6_AVTHT_pcl1', 'q6.7_AVSIT_pcl1', 'q6.8_AMNES_pcl1', 'q6.9_DISINT_pcl1',
                    'q6.10_DTACH_pcl1', 'q6.11_NUMB_pcl1', 'q6.12_FUTRE_pcl1', 'q6.13_SLEEP_pcl1',
                    'q6.14_ANGER_pcl1', 'q6.15_CONC_pcl1', 'q6.16_HYPER_pcl1', 'q6.17_STRTL_pcl1']

In [6]:
discrete_features = ["highschool_diploma", "dyslexia", "ADHD", "phq1", "lot1",
                    "trait1", "state1", "PCL1", "PCL_Broad1", "PCL_Strict1", "phq2", "lot2",
                    "trait2", "state2", "PCL2", "PCL_Broad2", "PCL_Strict2", "cd_risc1", "active_coping1",
                    "planning1", "positive_reframing1", "acceptance1", "humor1", "religion1",
                    "emotional_support1", "instrumental_support1", "self_distraction1", "denial1",
                    "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1", "active_coping2",
                    "planning2", "positive_reframing2", "acceptance2", "humor2", "religion2", "emotional_support2",
                    "instrumental_support2", "self_distraction2", "denial2", "venting2", "substance_use2",
                    "behavioral_disengagement2", "self_blame2", "trauma_history8_1",
                    'q6.1_INTRU_pcl1', 'q6.2_DREAM_pcl1', 'q6.3_FLASH_pcl1', 'q6.4_UPSET_pcl1',
                    'q6.5_PHYS_pcl1', 'q6.6_AVTHT_pcl1', 'q6.7_AVSIT_pcl1', 'q6.8_AMNES_pcl1', 'q6.9_DISINT_pcl1',
                    'q6.10_DTACH_pcl1', 'q6.11_NUMB_pcl1', 'q6.12_FUTRE_pcl1', 'q6.13_SLEEP_pcl1',
                    'q6.14_ANGER_pcl1', 'q6.15_CONC_pcl1', 'q6.16_HYPER_pcl1', 'q6.17_STRTL_pcl1',
                    'q6.1_INTRU_pcl2', 'q6.2_DREAM_pcl2', 'q6.3_FLASH_pcl2', 'q6.4_UPSET_pcl2',
                    'q6.5_PHYS_pcl2', 'q6.6_AVTHT_pcl2', 'q6.7_AVSIT_pcl2', 'q6.8_AMNES_pcl2', 'q6.9_DISINT_pcl2',
                    'q6.10_DTACH_pcl2', 'q6.11_NUMB_pcl2', 'q6.12_FUTRE_pcl2', 'q6.13_SLEEP_pcl2',
                    'q6.14_ANGER_pcl2', 'q6.15_CONC_pcl2', 'q6.16_HYPER_pcl2', 'q6.17_STRTL_pcl2']

In [7]:
genome_features = ["HML_5HTT", "HL_MAOA", "HML_NPY", "COMT_Hap1_recode",
                "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5"]
continuous_features = ["T1Acc1t", "T1Acc1n", "T1bias"]
demographic_features = ["highschool_diploma", "dyslexia", "ADHD", "Ashken_scale", "Sephar_scale", "Unknown_scale"]

In [8]:
df1 = df[features + ['q6.1_INTRU', 'q6.2_DREAM', 'q6.3_FLASH', 'tred_cutoff']]
#df1 = df1.dropna(thresh=47)
df_t1 = df1[t1_features]
df_t2 = df1[t2_features]
df1 = df1[(df_t1.isna().astype(int).sum(axis=1)<38) | (df_t2.isna().astype(int).sum(axis=1)<38)]
df1["t1_missing"] = df_t1.isna().astype(int).sum(axis=1)==38
df1["t2_missing"] = df_t2.isna().astype(int).sum(axis=1)==38
features.extend(["t1_missing", "t2_missing"])


In [9]:
X, X_out, Y, y_out = train_test_split(df1[features], df1[['q6.1_INTRU', 'q6.2_DREAM', 'q6.3_FLASH', 'tred_cutoff']],
                                      test_size=0.25, random_state=271828, stratify=df1['tred_cutoff'])

In [10]:
impute = 1
if impute:
    mice = IterativeImputer()
    X = pd.DataFrame(mice.fit_transform(X), columns=X.columns)
    Y = Y.fillna(Y.mean())
    features.extend(["outliers_count_t1_35", "outliers_count_t2_35", "outliers_count_t1_25", "outliers_count_t2_25"])
    X["outliers_count_t1_35"] = (np.abs(stats.zscore(X[t1_features])) > 3.5).sum(axis=1)
    X["outliers_count_t1_25"] = (np.abs(stats.zscore(X[t1_features])) > 2.5).sum(axis=1)
    X["outliers_count_t2_35"] = (np.abs(stats.zscore(X[t2_features])) > 3.5).sum(axis=1)
    X["outliers_count_t2_25"] = (np.abs(stats.zscore(X[t2_features])) > 2.5).sum(axis=1)
    
    

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, stratify=Y['tred_cutoff'], random_state=271828)
cv = StratifiedKFold(3, random_state=271828)
scores_f = []
scores_p = []
scores_r = []
scores_auc = []

for train, test in cv.split(X_train, y_train['tred_cutoff'].astype(int)):

    X_train_cv = X_train.iloc[train]
    y_train_cv = y_train.iloc[train]

    t = [y_train_cv['q6.1_INTRU'], y_train_cv['q6.2_DREAM'],
         y_train_cv['q6.3_FLASH'], y_train_cv['tred_cutoff']]
    
    y_pred_1 = XGBClassifier().fit(X_train_cv, (t[0]>2).values.astype(int)).predict_proba(X_train.iloc[test])
    y_pred_2 = XGBClassifier().fit(X_train_cv, (t[1]>2).values.astype(int)).predict_proba(X_train.iloc[test])
    y_pred_3 = XGBClassifier().fit(X_train_cv, (t[2]>2).values.astype(int)).predict_proba(X_train.iloc[test])
    y_pred_target = XGBClassifier().fit(X_train_cv, t[3]).predict_proba(X_train.iloc[test])

    y_pred = (y_pred_1[:, 1] + y_pred_2[:, 1] +  y_pred_3[:, 1]) > 0.5 #+ 0.5 * y_pred_target[:, 1]) > 1

    scores_f.append(f1_score(y_train['tred_cutoff'].iloc[test], y_pred))
    scores_p.append(precision_score(y_train['tred_cutoff'].iloc[test], y_pred))
    scores_r.append(recall_score(y_train['tred_cutoff'].iloc[test], y_pred))
    scores_auc.append(roc_auc_score(y_train['tred_cutoff'].iloc[test], y_pred))

In [ ]:
print("\nf1\n",scores_f, "\nmean\n", sum(scores_f)/len(scores_f))
print("\nprecision\n",scores_p, "\nmean\n", sum(scores_p)/len(scores_p))
print("\nrecall\n",scores_r, "\nmean\n", sum(scores_r)/len(scores_r))
print("\nroc auc\n",scores_auc, "\nmean\n", sum(scores_auc)/len(scores_auc))

In [16]:
? CatBoostClassifier

In [11]:
model = Pipeline(steps=[
    
   # ('anova', SelectKBest()),
    ('clf',  CatBoostClassifier(random_state=271828, verbose=0))])

grid_params = [{#'anova__k':[10, 20, 30, 40, 50,60, 70,80,90, 100],
    'clf__depth':[1,5,10],
          'clf__iterations':[100,500, 1000],
          'clf__learning_rate':[0.001, 0.1, 0.3],
          'clf__l2_leaf_reg':[1, 10, 100],
          #'clf__border_count':[32,5,10,20,50,100,200],
          #'ctr_leaf_count_limit':[50,5,10,20,100,200],
    'clf__scale_pos_weight':[0.1, 1, 10]}]


In [12]:
#xgb = XGBClassifier()
scv = StratifiedKFold(5)
gs_3 = GridSearchCV(model, grid_params,  scoring="roc_auc", cv=scv)
gs_3.fit(X[features], (Y['q6.3_FLASH']>2).values.astype(int))
print("roc_auc", gs_3.best_score_,"\nparams = ", gs_3.best_params_)

CatBoostError: bad allocation

In [ ]:
#xgb = XGBClassifier()
scv = StratifiedKFold(5)
gs_2 = GridSearchCV(model, grid_params,  scoring="roc_auc", cv=scv)
gs_2.fit(X[features], (Y['q6.2_DREAM']>2).values.astype(int))
print("roc_auc", gs_2.best_score_,"\nparams = ", gs_2.best_params_)

In [ ]:
#xgb = XGBClassifier()
scv = StratifiedKFold(5)
gs_1 = GridSearchCV(model, grid_params,  scoring="roc_auc", cv=scv)
gs_1.fit(X[features], (Y['q6.1_INTRU']>2).values.astype(int))
print("roc_auc", gs_1.best_score_,"\nparams = ", gs_1.best_params_)

In [ ]:
#xgb = XGBClassifier()
scv = StratifiedKFold(5)
tred_gs = GridSearchCV(model, grid_params,  scoring="roc_auc", cv=scv)
tred_gs.fit(X[features], (Y['tred_cutoff']).values.astype(int))
print("roc_auc", tred_gs.best_score_,"\nparams = ", tred_gs.best_params_)

In [ ]:

from sklearn.metrics import roc_curve, auc

#y_pred_only_avoidance = only_avoidance_gs.best_estimator_.predict_proba(X_test)
#y_pred_depression = depression_gs.best_estimator_.predict_proba(X_test)
#y_pred_tred = tred_gs.best_estimator_.predict_proba(X_test)
#y_pred_33 = regression_33_gs.best_estimator_.predict_proba(X_test)
y_pred_50 = regression_50_gs.best_estimator_.predict_proba(X_test)
y_pred_intrusion = intrusion_gs.best_estimator_.predict_proba(X_test)
y_pred_avoidance = avoidance_gs.best_estimator_.predict_proba(X_test)
y_pred_hypertension = hypertention_gs.best_estimator_.predict_proba(X_test)
preds = y_pred_50[:, 1]* y_pred_intrusion[:,1] * y_pred_avoidance[:,1] * y_pred_hypertension[:,1]

fpr, tpr, threshold = roc_curve(y_test, preds)
roc_auc = auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
plt.hist(preds[y_test==0], bins=50, color="b", label="non PTSD")
plt.hist(preds[y_test==1], color="r", label="PTSD")
plt.xlabel('pred probability')
plt.legend();
sorted(preds[y_test==1])